In [6]:
import random
from datetime import datetime

def get_current_weather(arguments):
    weather_conditions = ['晴天', '多云', '雨天']
    random_weather = random.choice(weather_conditions)
    location = arguments['location']
    return f"{location}今天是{random_weather}。"

def get_current_time():
    current_datetime = datetime.now()
    formatted_time = current_datetime.strftime('%Y-%m-%d %H:%M:%S')
    return f"当前时间：{formatted_time}"

print(get_current_weather({"location": "上海"}))
print(get_current_time())

上海今天是雨天。
当前时间：2025-04-05 21:27:13


In [7]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。"
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想知道指定城市的天气时非常有用。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                    }
                },
                "required": ["location"]
            }
        }
    }
]

tool_name = [tool["function"]["name"] for tool in tools]
print(f"创建了{len(tools)}个工具，为：{tool_name}\n")

创建了2个工具，为：['get_current_time', 'get_current_weather']



In [8]:
messages = [
    {
        "role": "system",
        "content": """你是一个很有帮助的助手。如果用户提问关于天气的问题，请调用 ‘get_current_weather’ 函数;
     如果用户提问关于时间的问题，请调用‘get_current_time’函数。
     请以友好的语气回答问题。"""
    },
    {
        "role": "user",
        "content": "上海天气"
    }
]

In [18]:
from openai import OpenAI
import os

client = OpenAI(
    api_key = os.getenv("DASHSCOPE_API_KEY"),
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1",
)

def function_call():
    completion = client.chat.completions.create(
        model = "qwen-plus",
        messages = messages,
        tools = tools,
    )
    print("返回对象: ")
    print(completion.choices[0].message.model_dump_json())
    print("\n")
    return completion

print("正在发起function calling...")    
completion = function_call()

正在发起function calling...
返回对象: 
{"content":"","refusal":null,"role":"assistant","audio":null,"function_call":null,"tool_calls":[{"id":"call_5c05a3762efa46c9bdd328","function":{"arguments":"{\"location\": \"上海\"}","name":"get_current_weather"},"type":"function","index":0}]}




In [19]:
import json

print("正在执行工具函数...")

function = completion.choices[0].message.tool_calls[0].function
function_name = function.name
function_arguments = function.arguments

arguments = json.loads(function_arguments)

function_mapper = {
    'get_current_time': get_current_time,
    'get_current_weather': get_current_weather
}

function = function_mapper[function_name]
if arguments == {}:
    function_output = function()
else:
    function_output = function(arguments)

print(f"工具函数输出：{function_output}")    

正在执行工具函数...
工具函数输出：上海今天是雨天。
